# Project Assignmnet 1 
***
#### Name = NIRAL PATEL
#### ID = 1303276
#### Date = 10/17/2022
#### Description = Decision Tree and Random Forest Classification model
***
## Reporting Tasks:
1. Compare the accuracies of the Random Forest classifier as a function of the number of base learners (e.g., 10, 50, 100, 500, 1000, and 500) and the number of features to consider at each split (e.g., auto or sqrt). Report your observations/conclusions and provide evidence to support your conclusions. [50 points]
2. Compare of the results of all the classifiers (with the best possible parameter setting for each classifier). Use classification accuracy (# of instances correctly classified/total # of instances presented for classification), per class classification accuracy, and confusion matrix to compare the classifiers. [50 points]


In [57]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

#Other additional imports
warnings.filterwarnings('ignore') 

In [58]:
df = pd.read_csv("spam.csv")

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 58 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   make        4601 non-null   float64
 1   address     4601 non-null   float64
 2   all         4601 non-null   float64
 3   3d          4601 non-null   float64
 4   our         4601 non-null   float64
 5   over        4601 non-null   float64
 6   remove      4601 non-null   float64
 7   internet    4601 non-null   float64
 8   order       4601 non-null   float64
 9   mail        4601 non-null   float64
 10  receive     4601 non-null   float64
 11  will        4601 non-null   float64
 12  people      4601 non-null   float64
 13  report      4601 non-null   float64
 14  addresses   4601 non-null   float64
 15  free        4601 non-null   float64
 16  business    4601 non-null   float64
 17  email       4601 non-null   float64
 18  you         4601 non-null   float64
 19  credit      4601 non-null  

#### Train the classifiers using the first 1000 instances and use the remaining 3601 for testing. Feel free to create separate training and testing data files. 
#### Have your own strategy to deal with any missing feature values (e.g., remove instances with missing features or fill in the missing feature values with the most popular value.).

In [60]:
# Creatae the Test and Train Data using basic python sysntex 
train_df = df.iloc[:1000,:]
test_df = df.iloc[1000:,:]

X_train = train_df.drop(columns='Class')
Y_train = train_df['Class']
X_test = test_df.drop(columns='Class')
Y_test = test_df['Class']
print("Total Train Data", train_df.shape,'\nTotal Test Data',test_df.shape,"\nTest Data set x:",X_test.shape,'\nTest Data set y:',Y_test.shape,'\nTrain Data set x:',X_train.shape,'\nTrain data set y:',Y_train.shape)

Total Train Data (1000, 58) 
Total Test Data (3601, 58) 
Test Data set x: (3601, 57) 
Test Data set y: (3601,) 
Train Data set x: (1000, 57) 
Train data set y: (1000,)


In [61]:
# Creatae the Test and Train Data using model selection
#--------------------------------
from sklearn.model_selection import train_test_split
#--------------------------------
X = df.drop(columns='Class')
y = df['Class']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.782)
print("Total Data", df.shape,"\nTest Data set x:",x_test.shape,'\nTest Data set y:',y_test.shape,'\nTrain Data set x:',x_train.shape,'\nTrain data set y:',y_train.shape)

Total Data (4601, 58) 
Test Data set x: (3598, 57) 
Test Data set y: (3598,) 
Train Data set x: (1003, 57) 
Train data set y: (1003,)


In [62]:
# Decision Tree classifier
#--------------------------
from sklearn.tree import DecisionTreeClassifier
#--------------------------

# Fit the training model to the desired classifier
decision_tree_clf = DecisionTreeClassifier()
decision_tree_clf = decision_tree_clf.fit(x_train, y_train)
dt_accuracy = decision_tree_clf.score(x_test, y_test)
print("Classification Accuracy: ", round((dt_accuracy),4), "\n")

Classification Accuracy:  0.876 



In [63]:
# Metrics - Decision Tree classifier
dt_predict = decision_tree_clf.predict(x_test)
print("Decision Tree classifier\n")
print ("Prediction:",dt_predict,'\n')
print("Accuracy:", round((accuracy_score(y_test, dt_predict)),4), "\n")
print("Report: \n", classification_report(y_test, dt_predict)) 
print("Confusion Matrix: \n", confusion_matrix(y_test, dt_predict), "\n")

Decision Tree classifier

Prediction: ['ham' 'ham' 'ham' ... 'ham' 'spam' 'spam'] 

Accuracy: 0.876 

Report: 
               precision    recall  f1-score   support

         ham       0.89      0.90      0.90      2175
        spam       0.85      0.84      0.84      1423

    accuracy                           0.88      3598
   macro avg       0.87      0.87      0.87      3598
weighted avg       0.88      0.88      0.88      3598

Confusion Matrix: 
 [[1960  215]
 [ 231 1192]] 



#### 1. Compare the accuracies of the Random Forest classifier as a function of the number of base learners (e.g., 10, 50, 100, 500, 1000, and 500) and the number of features to consider at each split (e.g., auto or sqrt). Report your observations/conclusions and provide evidence to support your conclusions. [50 points]

In [64]:
# Random Forest classifier
#--------------------------
from sklearn.ensemble import RandomForestClassifier
#--------------------------

estimator = [10,50,100,500,1000,5000] #estimate values
features = ['auto', 'sqrt'] #feature values
rf_accuracy_auto = [] # Empty array to store the Accuracy score
rf_accuracy_sqrt = []
print('Random Forest Classification model','\n')
for i in features: #for loop for feature selection
    print('Random Forest model with Feature:',i,'\n')
    for j in estimator: #for loop for estimator selection
        random_forest_clf = RandomForestClassifier(n_estimators=j, max_features=i) #create a RF model with feature and estimator
        random_forest_clf = random_forest_clf.fit(X_train, Y_train) #Train the model
        rf_score = random_forest_clf.score(X_test, Y_test) # Test the model and get the accuracy score
        print('Classification Accuracy with ',j,' estimators:', round((rf_score),4), "\n")
        if i == 'sqrt':
            rf_accuracy_sqrt.append(rf_score) #store the accuracy score if feature is squrt
        else:
            rf_accuracy_auto.append(rf_score) #store the accuracy score if feature is auto

accuracy1 = pd.DataFrame(rf_accuracy_sqrt,columns = ['accuracy_sqrt'])
accuracy2 = pd.DataFrame(rf_accuracy_auto,columns = ['accuracy_auto'])
accuracy = pd.merge(accuracy1,accuracy2,left_index=True,right_index=True)
print (accuracy)

Random Forest Classification model 

Random Forest model with Feature: auto 

Classification Accuracy with  10  estimators: 0.9284 

Classification Accuracy with  50  estimators: 0.9342 

Classification Accuracy with  100  estimators: 0.9309 

Classification Accuracy with  500  estimators: 0.9345 

Classification Accuracy with  1000  estimators: 0.9334 

Classification Accuracy with  5000  estimators: 0.9317 

Random Forest model with Feature: sqrt 

Classification Accuracy with  10  estimators: 0.9242 

Classification Accuracy with  50  estimators: 0.935 

Classification Accuracy with  100  estimators: 0.9334 

Classification Accuracy with  500  estimators: 0.9334 

Classification Accuracy with  1000  estimators: 0.9339 

Classification Accuracy with  5000  estimators: 0.9336 

   accuracy_sqrt  accuracy_auto
0       0.924188       0.928353
1       0.935018       0.934185
2       0.933352       0.930853
3       0.933352       0.934463
4       0.933907       0.933352
5       0.933630  

In [65]:
accuracy.describe()

,accuracy_sqrt,accuracy_auto
count,6.000000,6.000000
mean,0.932241,0.932148
std,0.003993,0.002334
min,0.924188,0.928353
25%,0.933352,0.931061
50%,0.933491,0.932519
75%,0.933838,0.933977
max,0.935018,0.934463


In [66]:
# Metrics - Random Forest classifier 
rf_predict = random_forest_clf.predict(X_test)

print("Random Forest classifier with SQRT and 5000  estimators\n")
print ("Prediction:",rf_predict,'\n')
print("Accuracy:", round((accuracy_score(Y_test, rf_predict)),4), "\n")
print("Report: \n", classification_report(Y_test, rf_predict)) 
print("Confusion Matrix: \n", confusion_matrix(Y_test, rf_predict), "\n")

Random Forest classifier with SQRT and 5000  estimators

Prediction: ['ham' 'ham' 'spam' ... 'ham' 'ham' 'spam'] 

Accuracy: 0.9336 

Report: 
               precision    recall  f1-score   support

         ham       0.93      0.96      0.95      2182
        spam       0.94      0.89      0.91      1419

    accuracy                           0.93      3601
   macro avg       0.93      0.93      0.93      3601
weighted avg       0.93      0.93      0.93      3601

Confusion Matrix: 
 [[2095   87]
 [ 152 1267]] 

